In [6]:
import preprocessing
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import project_preprocessing_helper
import gc

In [2]:
df = preprocessing.read_application()

In [3]:
previous_app = preprocessing.read_previous_application()

In [4]:
bureau = preprocessing.read_bureau()

In [7]:
df = df.join(bureau, on='SK_ID_CURR', how='left')
del bureau
gc.collect()

3180

In [8]:
df = df.join(previous_app, on='SK_ID_CURR', how='left')
del previous_app
gc.collect()

0

In [10]:
train, test, target, features_name = preprocessing.split_train_test_target(df)

In [17]:
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline

In [18]:
pipe_ada = Pipeline([('imputer', Imputer(strategy = 'median')),
                    ('scaler', StandardScaler()),
                    ('clf', AdaBoostClassifier(n_estimators=100))
                    ])

In [19]:
pipe_ada.fit(train, target)

Pipeline(memory=None,
         steps=[('imputer',
                 Imputer(axis=0, copy=True, missing_values='NaN',
                         strategy='median', verbose=0)),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                                    learning_rate=1.0, n_estimators=100,
                                    random_state=None))],
         verbose=False)

In [22]:
submit = pipe_ada.predict_proba(test)[:,1]

In [26]:
submit = test[['SK_ID_CURR']]
submit['TARGET'] = pipe_ada.predict_proba(test)[:,1]
submit.to_csv('submit_app_previous_bureau_ada.csv', index=False)

In [27]:
submit

,SK_ID_CURR,TARGET
0,100001,0.492535
1,100005,0.494209
2,100013,0.487122
3,100028,0.491878
4,100038,0.495423
...,...,...
48739,456221,0.493230
48740,456222,0.493018
48741,456223,0.489187
48742,456224,0.493423


In [41]:
feature_importance = pipe_ada.steps[2][1].feature_importances_

In [42]:
feature_importances = pd.DataFrame({'feature': features_name, 'importance': feature_importance})

In [43]:
feature_importances.sort_values(by='importance', ascending=False)[:20]

,feature,importance
110,RATIO_ANNUITY_CREDIT,0.10
35,EXT_SOURCE_2,0.09
36,EXT_SOURCE_3,0.09
22,DAYS_EMPLOYED,0.04
0,AMT_ANNUITY,0.03
34,EXT_SOURCE_1,0.03
273,bureau_AMT_CREDIT_SUM_DEBT_mean,0.03
366,previous_app_DAYS_LAST_DUE_1ST_VERSION_max,0.03
2,AMT_GOODS_PRICE,0.03
92,REGION_RATING_CLIENT_W_CITY,0.02
